In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as cons
import scipy

import time

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from qutip import *
import cmath

In [2]:
def wiginput(q, p, psi, method='iterative'):
    """
    Function to pass through 'x' and 'y' coordinates to the qutip Wigner function
    in order to use the scipy integrate package to evaluate the Wigner function for 
    input states
    """
    return float(wigner(psi, q, p, method=method))

def wigneginput(q, p, psi, method='iterative'):
    """
    Function to pass through 'x' and 'y' coordinates to the qutip Wigner function
    in order to use the scipy integrate package to evaluate the Wigner Logarithmic
    Negativity for input states
    """
    return abs(float(wigner(psi, q, p, method=method)))

# Photon added/subtracted Gaussian States

In [3]:
N=5 # Number of Fock levels used in calculations (dimensions of vectors and matrices)

sqf=0.8 # Squeezing Factor
dif=1 # Displacement Factor

In [ ]:
vac = basis(N, 0) # Vacuum state
coh = coherent(N, dif) # Coherent state
fockone = basis(N,1) # First Fock state

In [ ]:
# normalisation factor for the photon added state
Nadd = 1 + np.sinh(sqf) ** 2 + np.abs(dif) ** 2 

# state vector and density matrix for the photon added state
psiadd = (1/np.sqrt(Nadd)) * create(N) * displace(N,dif) * squeeze(N, sqf) * vac
rhoadd = ket2dm(psiadd)

In [ ]:
# normalisation factor for the photon subtracted state
Nsub = np.sinh(sqf) ** 2 + np.abs(dif) ** 2 

# state vector and density matrix for the photon subtracted state
psisub = (1/np.sqrt(Nsub)) *  destroy(N) * displace(N,dif) * squeeze(N, sqf) * vac
rhosub = ket2dm(psisub)

In [ ]:
# state vector and density matrix for the displaced squeeze state
psi =  displace(N,dif) * squeeze(N, sqf) * vac 

In [4]:
inputstate = fockone
starttime = time.time()
w = scipy.integrate.dblquad(wiginput, -np.inf, np.inf, lambda x: -np.inf, lambda x: np.inf, args=([inputstate]), epsabs=1e-3)
endtime = time.time()
print(w[0], 'time taken = ', endtime - starttime)

with open('Adaptive_WLN_Results', 'a') as f:
    print('integral of Wigner function for state {} = {}'.format(input('type name of state'),wn[0]), 'time taken = {}'.format(endtime - starttime), file = f)

0.9999999761288658 time taken =  20.38804841041565


In [ ]:
inputstate = fockone
starttime = time.time()
wn = scipy.integrate.dblquad(wigneginput, -np.inf, np.inf, lambda x: -np.inf, lambda x: np.inf, args=([inputstate]), epsabs=1e-3)
endtime = time.time()
print(wn[0], np.log2(wn[0]), 'time taken = ', endtime - starttime)

with open('Adaptive_WLN_Results', 'a') as f:
    print('integral of abs(Wigner) for state {} = {}'.format(input('type name of state'),wn[0]), '\n WLN (base 2) = {}'.format(np.log2(wn[0])), 'time taken = ', endtime - starttime, file = f)

# Cat states

In [6]:
def catstate(alpha, phi, theta, N):
    """
    Function to generate a cat state using the qutip module
    ... ++ variable descriptions
    """
    coh1 = np.cos(phi) * coherent(N, alpha)
    coh2 = np.sin(phi) * cmath.rect(1,theta) * coherent(N, -alpha)
    K = 1 + np.sin(2 * phi) * np.cos(theta) * np.exp(-2 * alpha * np.conj(alpha))
    norm = 1/np.sqrt(K)
    return norm * (coh1 + coh2)

In [8]:
alpha = 2
phi = np.pi / 4
theta = 0
N = 20


catpsi = catstate(alpha, phi, theta, N)

starttime = time.time()
wncat = scipy.integrate.dblquad(wiginput, 0, np.inf, lambda x: 0, lambda x: np.inf, args=([catpsi]), epsabs = 1e-4)
endtime = time.time()
print(wncat[0], 'time taken = ', endtime - starttime)

with open('Adaptive_WLN_Results', 'a') as f:
    print('integral of Wigner function for cat state {} = {}'.format(input(''),wn[0]), 'time taken = {}'.format(endtime - starttime), file = f)

0.24999780154813128 time taken =  133.5095067024231


In [ ]:
norm = 1 / wncat[0]
norm 

In [ ]:
np.log2(wncat[0])

# Conclusion
Adaptive integration is far too slow for our needs, try some sampled integration methods (another notebook)